In [32]:
import sympy
sympy.init_printing()

One thing Casadi is not good at currently is simplification, we rely on Sympy to check that the group is affine.

In [33]:
def g(name):
    """define the matrix lie group, inherits product from matrix"""
    R = sympy.Matrix(sympy.symbols('R^{:s}_0:9'.format(name))).reshape(3, 3)
    v = sympy.Matrix(sympy.symbols('v^{:s}_0:3'.format(name))).reshape(3, 1)
    x = sympy.Matrix(sympy.symbols('x^{:s}_0:3'.format(name))).reshape(3, 1)
    X = sympy.Matrix.vstack(sympy.Matrix.hstack(R, v, x),
                        sympy.Matrix([0, 0, 0, 1, 0]).T,
                        sympy.Matrix([0, 0, 0, 0, 1]).T)
    return X


def skew(x):
    """skew matrix"""
    M = sympy.zeros(3, 3)
    M[0, 1] = -x[2]
    M[0, 2] = x[1]
    M[1, 0] = x[2]
    M[1, 2] = -x[0]
    M[2, 0] = -x[1]
    M[2, 1] = x[0]
    return M


def f(X, u):
    """define the derivative of the group"""
    R = X[0:3, 0:3]
    g = sympy.Matrix([0, 0, sympy.symbols('g_0')])
    a = sympy.Matrix(u[0:3])
    omega = sympy.Matrix(u[3:6])
    v = X[:3, 3]
    x = X[:3, 4]
    dX = sympy.zeros(5, 5)
    dX[:3, :3] = R*skew(omega)
    # it is interesting to investigate what is still group
    # affine here, as long as you don't put a state dependent
    # value on the right tangent plane (multipliy by R), you are fine
    dX[:3, 3] = R*a + g
    #dX[:3, 3] = R*a + g + R*v # this will fail
    #dX[:3, 3] = R*a + g + R*g # this is fine
    dX[:3, 4] = v
    #dX[:3, 4] = R*v # this will fail
    return dX


a = sympy.Matrix(sympy.symbols('a_0:3'))
omega = sympy.Matrix(sympy.symbols('omega_0:3'))
u = sympy.Matrix.vstack(a, omega)
g_a = g('a')
g_b = g('b')
g_I = sympy.eye(5)
f(g_a, u)


def group_affine(f, g, verbose=False):
    res = sympy.simplify(f(g_a*g_b, u) - (f(g_a, u)*g_b + g_a*f(g_b, u) - g_a*f(g_I, u)*g_b))
    affine = sympy.Matrix.norm(res) == 0
    if verbose and not affine:
        print(res)
    return affine

In [34]:
group_affine(f, g, verbose=True)

True

The take away here, is that you can do anything you want in the tanget plane at the origin, if you are using left invaraint error, because both dynamics are impacted the same, since the rotation is zero. However, in the tangent plane after the rotation, if you have any non-constant dynamics, they will impact the error dynamics and the error equations will not be autonomous.